# Probing USR Dynamics with $\eta$

## Naming Clarification
In this codebase, there are two related parameters often called 'eta':
1. **`etaH` (in the code)**: This is the Hubble friction parameter $\delta_H = -\ddot{\phi}/(H\dot{\phi})$. In the Ultra-Slow Roll (USR) regime where $V' \approx 0$, friction dominates and $\ddot{\phi} \approx -3H\dot{\phi}$, so **`etaH` $\to$ 3**.
2. **$\eta$ (Slow-Roll parameter)**: Defined as $d \ln \epsilon / dN$. This is the parameter that **reaches -6** in USR. 

We use the identity $\eta = 2(\epsilon_H - \delta_H)$ to calculate the parameter that benchmarks the USR plateau at -6.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

# Ensure we can import from root directory
sys.path.append(os.path.abspath('..'))

from inflation_models import HiggsModel
from inf_dyn_background import run_background_simulation, get_derived_quantities

def detect_usr_duration(N, eta, center=-6.0, width=1.0):
    """
    Detects the duration of the USR phase where the slow-roll eta is within 
    a window around the USR benchmark value (typically -6).
    """
    lower = center - width
    upper = center + width
    usr_mask = (eta > lower) & (eta < upper)
    
    if not np.any(usr_mask):
        return 0.0, None, None
    
    indices = np.where(usr_mask)[0]
    start_idx = indices[0]
    end_idx = indices[-1]
    
    delta_N = N[end_idx] - N[start_idx]
    return delta_N, N[start_idx], N[end_idx]

## 1. CMB-Scale USR Analysis

To trigger a USR phase on the plateau ($\phi \approx 5.42$), the initial velocity must be high enough to "overshoot" the standard slow-roll attractor velocity ($y_{SR} \approx -0.003$). 

We use the discovered fine-tuned velocity for this scale:
- **$\phi_i = 5.42$**
- **$v_i = -0.1$** (Large enough to trigger friction-dominated deceleration while still potential-dominated)

In [ ]:
# Setup Model with Refined CMB-Scale Fine-Tuning
model = HiggsModel()
model.xi = 5.42
model.yi = -0.1 # Velocity overshoot

# Run Simulation
T_span = np.linspace(0, 50, 2000) # Short run to focus on the initial USR
sol = run_background_simulation(model, T_span)
data = get_derived_quantities(sol, model)

# Calculate the slow-roll parameter eta (reaches -6)
N = data['N']
eta_tot = 2 * (data['epsH'] - data['etaH'])

# Detect USR Phase
duration, n_start, n_end = detect_usr_duration(N, eta_tot)
print(f"Detected USR Duration: {duration:.4f} e-folds")
if n_start is not None:
    print(f"USR Window: N = [{n_start:.3f}, {n_end:.3f}]")

## 2. Visualization

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.set_xlabel('e-folds (N)')
ax1.set_ylabel('Parameters')
ax1.plot(N, eta_tot, label=r'$\eta \equiv d \ln \epsilon / dN$', color='blue', lw=2)
ax1.plot(N, data['etaH'], label='etaH (Hubble friction)', color='green', linestyle='--')
ax1.axhline(-6, color='black', linestyle=':', alpha=0.5)
ax1.axhline(3, color='black', linestyle=':', alpha=0.5)

if n_start is not None:
    ax1.axvspan(n_start, n_end, color='gray', alpha=0.2, label='Fine-Tuned USR')

ax1.set_ylim(-7, 5)
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)

plt.title(f"Fine-Tuned USR at CMB Exit (phi_i={model.xi}, vi={model.yi})")
fig.tight_layout()
plt.show()